In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import re
import requests

In [75]:
proxy = "http://45.140.13.124:9137"

In [76]:
def getdriver(): #it is the selenium driver to connect the script with the browser
    options = webdriver.ChromeOptions()
    options.add_argument('log-level=3')
    options.add_argument("--window-size=1880x1020")
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    #options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("test-type")
    options.add_argument("--disable-web-security")
    options.add_argument("--allow-running-insecure-content")
    options.add_argument('--proxy-server=%s' % proxy)
    driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())
    #     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """})
    return driver
driver = getdriver()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Pfactorial00110\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_9648\3269282774.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())


In [70]:
job_link = []
Category = []
#ds_url = 'https://www.naukrigulf.com/data-science-jobs-'+str(i)+'?ts=1650635291000&requestsource=mr&recentSearch=false'
#ml_url = 'https://www.naukrigulf.com/machine-learning-jobs-'+str(i)+'?ts=1650635232000&requestsource=mr&recentSearch=true'

In [71]:
def carb_extractor():#this functions extracts the job links from the particular websie
  try:
    for i in range(0,40): # iterates upto 10 pages
      url_next='https://www.naukrigulf.com/machine-learning-jobs-'+str(i)+'?ts=1650635232000&requestsource=mr&recentSearch=true'
      driver.get(url_next) # to access the url
      content = driver.page_source
      page = BeautifulSoup(content, "html parser")
      for ele in page:
        a=ele.find('a')
        job_link.append(a['href']) # adds the iterted data to the empty list
        Category.append('Machine Learning')
  except:
    pass
  return job_link,Category

[]


In [17]:
job_link,Category = carb_extractor()

In [18]:
job_link

[]

In [68]:
Job_posting_df=pd.DataFrame({"Category":Category,"Job_Posting_link":job_link})

In [69]:
Job_posting_df.to_csv('Machine_Learning_naukri_links.csv', index=False)

In [77]:
DS_links = pd.read_csv("Data_science_naukri_links.csv")
ML_links = pd.read_csv("Machine_Learning_naukri_links.csv")

In [78]:
def carb_details_extractor(df): # this function extracts the data from the particular job link
    carbs_df = pd.DataFrame()
    for i in range(0,len(df)): #it iterates through the links upto the extracted links length
        try:
          Job_Posting_link=df['Job_Posting_link'][i] #iterates through the links in the dataframe
          Category=df['Category'][i]
          driver.get(Job_Posting_link)
          #content = driver.page_source
          #soups1=BeautifulSoup(content,'html.parser')
          try:
            Location=driver.find_element(by=By.XPATH, value='//*[@id="jobDescription"]/div[1]/section[1]/div[3]/div[1]/div[1]/div[2]/p[2]').text #extracts the text data from the respective class                                   #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[1]/p[2]
          except:
            Location=None
          try:
            company = driver.find_element(by=By.CLASS_NAME, value="info-org").text #soups1.find(class_ = 'info-org').text
          except:
            company=None
          try:
            title = driver.find_element(by=By.CLASS_NAME, value="info-position").text #soups1.find(class_ = 'info-position').text
          except:
            title=None
          try:
            salary = driver.find_element(by=By.CLASS_NAME, value='salary').text
          except:
            salary=None
          try:                                                    
            description = driver.find_element(by=By.XPATH, value='//*[@id="jobDescription"]/div[1]/section[1]/div[3]/article[1]/section/div').text #soups1.find(class_='job-description').text 
          except:
            description=None
          try:
            experience = driver.find_element(by=By.XPATH, value='//*[@id="jobDescription"]/div[1]/section[1]/div[3]/div[1]/div[1]/div[1]/p[2]').text
          except:
            experience=None
          try:
            education = driver.find_element(by=By.XPATH, value='//*[@id="jobDescription"]/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]').text #('//div[@class="col"][3]').text #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]
          except:
            education=None
          carbs=pd.DataFrame({'Category':Category,'Job_Posting_link':Job_Posting_link,'Job_title':title,'Company_name':company,'Location':Location,'Education':education,'Experience':experience,'Description':[description]},index=[0])
          carbs_df=carbs_df.append(carbs)
        except Exception as e: # raises an exception if something goes wrong in the time of extraction
          print(e)
    return carbs_df

In [79]:
DATA_SCIENCE = carb_details_extractor(DS_links) #calls the above funtion

C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_9648\2079402009.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_9648\2079402009.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_9648\2079402009.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_9648\2079402009.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  carbs_df=carbs_df.append(carbs)
C:\Users\Pfactorial00110

In [8]:
DATA_SCIENCE.to_csv('Naukri_Data_science_full_details_new.csv', index=False)

In [16]:
Machine_learning = carb_details_extractor(ML_links) #calls the above funtion

C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_7428\3668395011.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  Location=driver.find_element_by_xpath('//div[@class="col"][2]').text #extracts the text data from the respective class                                   #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[1]/p[2]
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_7428\3668395011.py:31: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  experience = driver.find_element_by_xpath('//div[@class="col"][1]').text
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_7428\3668395011.py:35: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  education = driver.find_element_by_xpath('//div[@class="col"][3]').text                                      #

In [18]:
Machine_learning.to_csv("naukri_Machine_learning_full_details.csv", index=False)